In [0]:
# [AUTENTICACIÒN CON TOKEN PERSONALIZADO] 
# ------------------------------------------------------------
# Configuración segura para montar ADLS Gen2 usando credenciales de Azure Active Directory
# ¡Reemplaza los valores entre { } según tu entorno!

# Configura el proveedor de token personalizado (integrado con Databricks)
configs = {
  "fs.azure.account.auth.type": "CustomAccessToken",  # Método de autenticación para Databricks passthrough
  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")  
  # Clase del proveedor automático de tokens (no modificar)
}

# [MONTAJE DE CONTENEDOR BRONZE]
# ------------------------------------------------------------
dbutils.fs.mount(
  source="abfss://bronze@forestsg.dfs.core.windows.net/",  # Formato: abfss://{nombre-contenedor}@{nombre-cuenta-almacenamiento}.dfs.core.windows.net/
  mount_point="/mnt/bronze",  # Ruta de montaje: /mnt/{nombre-directorio}
  extra_configs=configs  # Configuración de autenticación previa
)

# ------------------------------------------------------------
# [PARÁMETROS A MODIFICAR]
# 1. {nombre-contenedor}: Contenedor ADLS (bronze/silver/gold)
# 2. {nombre-cuenta-almacenamiento}: Nombre de tu Storage Account
# 3. {nombre-directorio}: Directorio local en DBFS (debe existir en /mnt/)

# [NOTAS DE SEGURIDAD]
# - Este método usa credenciales de Azure AD vinculadas al workspace de Databricks
# - Requiere permisos RBAC "Storage Blob Data Contributor" en el contenedor
# - No expone claves de acceso directamente en el código

# [COMANDOS ÚTILES]
# - Verificar montaje: display(dbutils.fs.ls("/mnt/bronze"))
# - Desmontar contenedor: dbutils.fs.unmount("/mnt/bronze")

True

In [0]:
# Solución: Verificar la existencia del path
dbutils.fs.exists("/mnt/bronze/SalesLT")  # Debe devolver True

[FileInfo(path='dbfs:/mnt/bronze/SalesLT/Address/', name='Address/', size=0, modificationTime=1739481175000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1739481176000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1739481175000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/Product/', name='Product/', size=0, modificationTime=1739481176000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1739481178000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1739481171000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1739481192000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1739481175000),
 FileInf

In [0]:
# [DESCRIPCIÓN] Montar un contenedor de Azure Data Lake Storage (ADLS) en Databricks
# ----------------------------------------------------------------------------------
# - Este código conecta Databricks con ADLS para acceder a datos procesados (ej: capa Silver)
# - ¡Reemplaza los valores específicos del entorno entre { } según tu configuración!

dbutils.fs.mount(
  source = "abfss://silver@forestsg.dfs.core.windows.net/",  # Formato: abfss://{nombre-contenedor}@{nombre-cuenta-almacenamiento}.dfs.core.windows.net/
  mount_point = "/mnt/silver",  # Ruta local en Databricks: /mnt/{nombre-directorio-montaje}
  extra_configs = configs  # Configuración de autenticación (detalles abajo)
)

# ----------------------------------------------------------------------------------
# [PARÁMETROS A MODIFICAR]
# 1. {nombre-contenedor}: Nombre del contenedor ADLS (ej: "bronze", "silver").
# 2. {nombre-cuenta-almacenamiento}: Nombre de tu Storage Account en Azure.
# 3. {nombre-directorio-montaje}: Directorio local en DBFS (debe empezar con "/mnt/").
#
# [NOTAS DE CONFIGURACIÓN]
# - "configs" debe ser un diccionario con credenciales de autenticación. Ejemplo seguro:
#   configs = {"fs.azure.account.key.{nombre-cuenta-almacenamiento}.dfs.core.windows.net": dbutils.secrets.get(scope="{key-vault-scope}", key="{nombre-secreto}")}
# - Usa Azure Key Vault para almacenar claves (¡nunca codifiques credenciales directamente!).
#
# [COMANDOS ÚTILES]
# - Verificar montaje: display(dbutils.fs.ls("/mnt/silver"))
# - Desmontar: dbutils.fs.unmount("/mnt/silver")

True

In [0]:
# [DESCRIPCIÓN] Montar la capa Gold (datos procesados) desde ADLS Gen2
# ----------------------------------------------------------------------
# - Este código monta el contenedor Gold donde se almacenan datos listos para reporting
# - ¡Reemplaza los valores entre { } según tu entorno!

dbutils.fs.mount(
  source = "abfss://gold@forestsg.dfs.core.windows.net/",  # Formato: abfss://{nombre-contenedor}@{nombre-cuenta-almacenamiento}.dfs.core.windows.net/
  mount_point = "/mnt/gold",  # Ruta local en Databricks: /mnt/{nombre-directorio}
  extra_configs = configs  # Configuración de autenticación (usando token AAD)
)

# ----------------------------------------------------------------------
# [PARÁMETROS A MODIFICAR]
# 1. {nombre-contenedor}: Nombre del contenedor ADLS (ej: "gold")
# 2. {nombre-cuenta-almacenamiento}: Nombre de tu Storage Account en Azure
# 3. {nombre-directorio}: Directorio local en DBFS (debe existir en /mnt/)

# [NOTAS DE CONFIGURACIÓN]
# - "configs" debe usar autenticación segura (recomendado: Azure AD Passthrough o Service Principal)
# - Ejemplo de configuración con Service Principal:
#   configs = {"fs.azure.account.auth.type": "OAuth",
#              "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
#              "fs.azure.account.oauth2.client.id": "{client-id}",
#              "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="{key-vault-scope}", key="{client-secret-key}"),
#              "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{tenant-id}/oauth2/token"}

# [COMANDOS ÚTILES]
# - Verificar datos procesados: display(dbutils.fs.ls("/mnt/gold"))
# - Desmontar: dbutils.fs.unmount("/mnt/gold")
# - Verificar autenticación: dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

True